# 📊 PDF 데이터시트 전처리 도구

**데이터시트 PDF를 분석하여 유용한 정보를 추출하고 JSON 형태로 변환합니다.**

---

## 🚀 시작하기 전에

이 노트북은 PDF 데이터시트를 처리하여 나중에 챗봇에서 사용할 수 있는 JSON 파일을 생성합니다.
분할 방식으로 PDF를 처리하여 더 효율적으로 정보를 추출합니다.

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: Google Drive 연동 { display-mode: "form" }
#@markdown Google Drive를 연동하여 PDF 파일을 불러오고 처리된 JSON 파일과 벡터 스토어를 저장합니다.

import os
import json
from pathlib import Path

# 환경 선택 토글
use_google_drive = True #@param {type:"boolean"}
#@markdown ✅ 체크: Google Drive 사용 (Colab 환경) | ❌ 체크해제: 로컬 환경 사용

# 환경별 설정
if use_google_drive:
    # Google Drive 환경
    print("🌐 Google Drive 환경으로 설정됩니다...")
    try:
        from google.colab import drive
        print("🔄 Google Drive 마운트 중...")
        drive.mount('/content/drive')
        
        # Google Drive 폴더 경로 설정
        base_path = Path('/content/drive/MyDrive')
        pdf_folder = base_path / 'datasheets'
        pre_json_folder = base_path / 'prep_json' / 'pre_json'
        result_json_folder = base_path / 'prep_json'
        vectorstore_folder = base_path / 'vectorstore'
        
        print("✅ Google Drive 연동 완료!")
        
    except ImportError:
        print("⚠️ Google Colab 환경이 아닙니다. 로컬 환경으로 전환합니다...")
        use_google_drive = False
        
if not use_google_drive:
    # 로컬 환경
    print("💻 로컬 환경으로 설정됩니다...")
    
    # 현재 작업 디렉토리 기준 로컬 폴더 설정 (./ 기준)
    base_path = Path.cwd()  # prep 폴더에서 ./ 기준으로 설정
    pdf_folder = base_path / 'datasheets'
    pre_json_folder = base_path / 'prep_json' / 'pre_json'
    result_json_folder = base_path / 'prep_json'
    vectorstore_folder = base_path / 'vectorstore'
    
    print("✅ 로컬 환경 설정 완료!")

# 폴더가 없으면 생성
pdf_folder.mkdir(parents=True, exist_ok=True)
pre_json_folder.mkdir(parents=True, exist_ok=True)
result_json_folder.mkdir(parents=True, exist_ok=True)
vectorstore_folder.mkdir(parents=True, exist_ok=True)

# param.json 파일 업데이트 (환경에 맞는 경로로)
param_file = Path('misc/param.json')
if param_file.exists():
    with open(param_file, 'r') as f:
        params = json.load(f)
    
    # 환경에 따른 경로 설정
    if use_google_drive:
        # Google Drive 경로
        base_path_str = '/content/drive/MyDrive'
        params['pdf_processing']['folders']['pdf_folder'] = f'{base_path_str}/datasheets'
        params['pdf_processing']['folders']['pre_json_folder'] = f'{base_path_str}/prep_json/pre_json'
        params['pdf_processing']['folders']['result_json_folder'] = f'{base_path_str}/prep_json'
        params['vectorstore']['folders']['vectorstore_folder'] = f'{base_path_str}/vectorstore'
    else:
        # 로컬 경로 (상위 디렉토리 기준)
        params['pdf_processing']['folders']['pdf_folder'] = './datasheets'
        params['pdf_processing']['folders']['pre_json_folder'] = './prep_json/pre_json'
        params['pdf_processing']['folders']['result_json_folder'] = './prep_json'
        params['vectorstore']['folders']['vectorstore_folder'] = './vectorstore'
    
    # 업데이트된 설정 저장
    with open(param_file, 'w') as f:
        json.dump(params, f, indent=4)
    
    print(f"✅ param.json이 {env_type} 환경에 맞게 업데이트되었습니다!")
    print(f"📋 설정된 경로들:")
    print(f"  • PDF 폴더: {params['pdf_processing']['folders']['pdf_folder']}")
    print(f"  • 중간 JSON 폴더: {params['pdf_processing']['folders']['pre_json_folder']}")
    print(f"  • 최종 JSON 폴더: {params['pdf_processing']['folders']['result_json_folder']}")
    print(f"  • 벡터스토어 폴더: {params['vectorstore']['folders']['vectorstore_folder']}")
else:
    print("⚠️ param.json 파일을 찾을 수 없습니다!")

# 설정된 폴더 경로 출력
env_type = "Google Drive" if use_google_drive else "로컬"
print(f"\n✅ {env_type} 환경 설정 완료!")
print(f"📁 PDF 폴더: {pdf_folder}")
print(f"📁 중간 JSON 폴더: {pre_json_folder}")
print(f"📁 최종 JSON 폴더: {result_json_folder}")
print(f"📁 벡터 스토어 폴더: {vectorstore_folder}")

# PDF 파일 목록 확인
pdf_files = list(pdf_folder.glob("*.pdf"))
print(f"\n�� 발견된 PDF 파일: {len(pdf_files)}개")
for pdf in pdf_files[:5]:  # 처음 5개만 표시
    print(f" - {pdf.name}")
    
if len(pdf_files) > 5:
    print(f" ... 외 {len(pdf_files) - 5}개")

if not pdf_files:
    print(f"\n⚠️ PDF 파일이 없습니다. '{pdf_folder}' 폴더에 PDF 파일을 업로드해주세요.")

In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
from pathlib import Path
import json

print("📥 필요한 라이브러리 설치 중...")

# GitHub 저장소 클론
!git clone https://github.com/doyoung42/chipchat_demo.git
%cd chipchat_demo/prep

# requirements.txt 설치
!pip install -r requirements.txt -q

# 현재 디렉토리 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# 설정 파일 로드
models_path = Path('misc/models.json')
param_path = Path('misc/param.json')

with open(models_path, 'r') as f:
    models = json.load(f)
with open(param_path, 'r') as f:
    params = json.load(f)

print("✅ 라이브러리 설치 완료!")
print("\n📋 사용 가능한 모델 목록:")
print("\nOpenAI 모델:")
for model in models["openai_models"]:
    print(f"• {model}")
print("\nClaude 모델:")
for model in models["claude_models"]:
    print(f"• {model}")

print("\n📋 현재 PDF 처리 설정:")
print(f"• 페이지당 청크 수: {params['pdf_processing']['pages_per_chunk']}")
print(f"• 출력 형식:")
print(f"  - 필터링된 PDF 저장: {params['pdf_processing']['output_formats']['save_filtered_pdf']}")
print(f"  - 카테고리별 시맨틱 청킹: 활성화됨")
print(f"  - 페이지 요약 및 카테고리 분류: 활성화됨")

In [ ]:
#@title 🔑 3단계: API 키 설정 { display-mode: "form" }
#@markdown API 키를 입력하고 사용할 LLM 모델을 선택합니다.

import os
import json
from pathlib import Path

# API 키 입력
openai_api_key = "" #@param {type:"string"}
claude_api_key = "" #@param {type:"string"}
hf_token = "" #@param {type:"string"}

# LLM 모델 선택
llm_model = "claude-3-sonnet-20240229" #@param ["claude-3-5-sonnet-20241022", "claude-3-opus-20240229", "claude-3-sonnet-20240229", "claude-3-haiku-20240307", "gpt-3.5-turbo-0125", "gpt-4o-mini-2024-07-18"] {type:"string"}

# API 키 유효성 검사
if not openai_api_key and not claude_api_key and not hf_token:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 최소 하나 이상의 API 키를 입력해주세요.")
    print("• OpenAI API 키: https://platform.openai.com/api-keys")
    print("• Claude API 키: https://console.anthropic.com/keys")
    print("• HuggingFace 토큰: https://huggingface.co/settings/tokens")
else:
    # key.json 파일에 저장
    key_path = Path('misc/key.json')
    key_path.parent.mkdir(exist_ok=True)
    
    # 모델 타입 결정
    if llm_model.startswith("claude"):
        provider = "claude"
    else:
        provider = "openai"
    
    with open(key_path, 'w') as f:
        json.dump({
            'openai_api_key': openai_api_key,
            'anthropic_api_key': claude_api_key,
            'huggingface_api_key': hf_token,
            'selected_models': {
                provider: llm_model
            }
        }, f, indent=2)
    
    print("✅ API 키와 선택된 모델이 key.json 파일에 저장되었습니다.")

In [ ]:
#@title 📊 4단계: PDF 파일 처리 설정 { display-mode: "form" }
#@markdown PDF 처리 설정을 구성합니다.

import json
from pathlib import Path

# 1단계에서 정의한 변수들이 존재하는지 확인
print("📋 1단계 변수 확인:")
try:
    print(f"• pdf_folder 존재: {pdf_folder}")
    print(f"• result_json_folder 존재: {result_json_folder}")
    print(f"• vectorstore_folder 존재: {vectorstore_folder}")
    print(f"• 현재 작업 디렉토리: {os.getcwd()}")
except NameError as e:
    print(f"❌ 변수 없음: {e}")
    print("💡 1단계를 먼저 실행해주세요!")

# param.json 파일 로드
param_path = Path('misc/param.json')
with open(param_path, 'r') as f:
    params = json.load(f)

# 현재 설정값 표시
print("📋 현재 PDF 처리 설정:")
print(f"• 페이지당 청크 수: {params['pdf_processing']['pages_per_chunk']}")
print(f"• 출력 형식:")
print(f"  - 필터링된 PDF 저장: {params['pdf_processing']['output_formats']['save_filtered_pdf']}")
print(f"  - 카테고리별 시맨틱 청킹: 활성화됨")
print(f"  - 페이지 요약 및 카테고리 분류: 활성화됨")

# 설정값 수정
pages_per_chunk = 3 #@param {type:"slider", min:1, max:10, step:1}
save_filtered_pdf = True #@param {type:"boolean"}
save_summary_only = True #@param {type:"boolean"} 
save_combined = True #@param {type:"boolean"}

# 설정값 업데이트
params["pdf_processing"]["pages_per_chunk"] = pages_per_chunk
params["pdf_processing"]["output_formats"]["save_filtered_pdf"] = save_filtered_pdf
params["pdf_processing"]["output_formats"]["save_summary_only"] = save_summary_only
params["pdf_processing"]["output_formats"]["save_combined"] = save_combined

# 폴더 경로 업데이트 (1단계에서 정의한 변수들 사용)
params["pdf_processing"]["folders"]["pdf_folder"] = str(pdf_folder)
params["pdf_processing"]["folders"]["pre_json_folder"] = str(result_json_folder / "pre_json")
params["pdf_processing"]["folders"]["result_json_folder"] = str(result_json_folder)
params["vectorstore"]["folders"]["vectorstore_folder"] = str(vectorstore_folder)

# 경로 검증
print(f"🔍 pdf_folder: {pdf_folder}")
print(f"🔍 result_json_folder: {result_json_folder}")
print(f"🔍 vectorstore_folder: {vectorstore_folder}")
print(f"🔍 param.json에 저장될 vectorstore 경로: {params['vectorstore']['folders']['vectorstore_folder']}")

# param.json 파일에 저장
param_path = Path('misc/param.json')
param_path.parent.mkdir(exist_ok=True)

with open(param_path, 'w') as f:
    json.dump(params, f, indent=2)

# 저장 후 확인
print(f"📝 param.json 파일 저장 위치: {param_path.absolute()}")
with open(param_path, 'r') as f:
    saved_params = json.load(f)
actual_vectorstore_path = saved_params.get('vectorstore', {}).get('folders', {}).get('vectorstore_folder', 'NOT_FOUND')
print(f"🔍 실제 저장된 vectorstore 경로: {actual_vectorstore_path}")

if actual_vectorstore_path != str(vectorstore_folder):
    print(f"⚠️ 경로 불일치 발견!")
    print(f"   예상: {vectorstore_folder}")
    print(f"   실제: {actual_vectorstore_path}")
else:
    print(f"✅ 경로 저장 성공!")

print("\n✅ PDF 처리 설정이 param.json 파일에 저장되었습니다.")
print("\n📝 설정 설명:")
print("• 페이지당 청크 수: PDF를 몇 페이지씩 묶어서 처리할지 설정합니다.")
print("• 필터링된 PDF 저장: 유용한 정보가 있는 페이지만 모아서 PDF로 저장합니다.")
print("• 카테고리별 처리: 내용을 다음 카테고리로 분류하여 처리합니다:")
print("  - 제품 요약(Product summary)")
print("  - 전기적 특성(Electrical Characteristics)")
print("  - 응용 회로 예시(Application circuits)")
print("  - 기계적 특성(Mechanical Characteristics)")
print("  - 신뢰성 및 환경 시험 조건(Reliability, Environmental conditions)")
print("  - 패키징 정보(Packaging)")

In [ ]:
#@title 📋 4-1단계: 부품 정보 CSV 파일 확인 { display-mode: "form" }
#@markdown PDF 폴더 내 pdf_filenames.csv 파일을 검사하고 부품 정보를 확인합니다.

import pandas as pd
import os
from pathlib import Path

# PDF 폴더 내 CSV 파일 경로
csv_path = os.path.join(pdf_folder, 'pdf_filenames.csv')

# CSV 파일 존재 여부 확인
if os.path.exists(csv_path):
    # CSV 파일 로드
    df = pd.read_csv(csv_path)
    
    # 필수 컬럼 확인
    required_columns = ['maker pn']
    optional_columns = ['fake_grade', 'final_fake_code']
    
    missing_columns = [col for col in required_columns if col not in df.columns]
    missing_optional = [col for col in optional_columns if col not in df.columns]
    
    print(f"✅ PDF 폴더에서 pdf_filenames.csv 파일을 찾았습니다.")
    print(f"📊 총 {len(df)} 개의 부품 정보가 있습니다.\n")
    
    # 컬럼 확인
    print("📋 CSV 파일 컬럼 정보:")
    for col in df.columns:
        print(f"• {col}")
    
    # 필수 컬럼 확인
    if missing_columns:
        print(f"\n⚠️ 필수 컬럼이 누락되었습니다: {', '.join(missing_columns)}")
        print("  특정 부품 번호 처리를 위해 'maker pn' 컬럼이 필요합니다.")
    
    # 옵션 컬럼 확인
    if missing_optional:
        print(f"\n🔍 옵션 컬럼이 누락되었습니다: {', '.join(missing_optional)}")
        print("  JSON 결과에 grade와 spec 정보를 추가하려면 'fake_grade'와 'final_fake_code' 컬럼이 필요합니다.")
    
    # 데이터 미리보기
    print("\n📄 데이터 미리보기:")
    display(df.head(5))
    
    # CSV 형식 설명
    print("\n📝 CSV 파일 형식 설명:")
    print("• 파일명 컬럼: PDF 파일명을 포함하는 컬럼")
    print("• maker pn: 데이터시트에서 찾을 특정 부품 번호")
    print("• fake_grade: JSON 결과에 추가될 등급 정보")
    print("• final_fake_code: JSON 결과에 추가될 사양 코드")

else:
    print(f"⚠️ PDF 폴더에서 pdf_filenames.csv 파일을 찾을 수 없습니다: {csv_path}")
    print("\n💡 CSV 파일 형식 예시:")
    print("파일명, maker pn, fake_grade, final_fake_code")
    print("example.pdf, ABC123, A, 12345")
    print("multi_component.pdf, XYZ789, B, 67890")
    
    # 예시 CSV 데이터 생성
    example_data = {
        'file': ['example1.pdf', 'example2.pdf', 'example3.pdf'],
        'maker pn': ['LM324', 'NE555', 'ATmega328'],
        'fake_grade': ['A', 'B', 'C'],
        'final_fake_code': ['12345', '67890', '54321']
    }
    
    example_df = pd.DataFrame(example_data)
    print("\n📊 CSV 파일 예시:")
    display(example_df)

In [ ]:
#@title 📄 5단계: PDF 파일 처리 { display-mode: "form" }
#@markdown PDF 파일들을 처리합니다.

from src.pdf_processor import PDFProcessor
from src.llm_manager import LLMManager
import json
from pathlib import Path

# key.json에서 선택된 모델 정보 로드
key_path = Path('misc/key.json')
with open(key_path, 'r') as f:
    keys = json.load(f)
selected_models = keys.get('selected_models', {})

# 선택된 모델에 따라 provider 결정
if any(model.startswith('claude') for model in selected_models.values()):
    provider = "claude"
    model_name = selected_models.get('claude')
else:
    provider = "openai"
    model_name = selected_models.get('openai')

# LLM Manager 초기화
llm_manager = LLMManager(provider=provider, model_name=model_name)

# PDF Processor 초기화
processor = PDFProcessor(llm_manager)

# PDF 처리 시작
print("🔄 PDF 파일 처리 시작...")
processor.process_pdf_folder()
print("✅ PDF 파일 처리 완료!")

In [ ]:
#@title 📊 6단계: 후처리 - JSON 병합 및 정규화 { display-mode: "form" }
#@markdown 처리된 JSON 파일들을 CSV 메타데이터와 병합하고 필드명을 정규화합니다.

from src.post_processor import PostProcessor
from pathlib import Path

# PostProcessor 초기화 (Google Drive 기본 경로 사용)
post_processor = PostProcessor()

print("🔄 후처리 파이프라인 시작...")
print("📋 처리 단계:")
print("  1. JSON 파일과 CSV 메타데이터 병합")
print("  2. 필드명 정규화 (filename → part number, fake_grade → grade)")
print("  3. 데이터셋 검증 및 정리")
print("  4. 처리되지 않은 PDF 파일 분리")

# 완전한 후처리 파이프라인 실행
try:
    post_processor.run_complete_postprocessing()
    
    # 처리 결과 요약
    print("\n📊 처리 결과 요약:")
    summary = post_processor.get_processing_summary()
    
    for key, value in summary.items():
        print(f"• {key.replace('_', ' ').title()}: {value}")
        
    print("\n✅ 후처리 파이프라인 완료!")
    print(f"📁 최종 JSON 파일 위치: {post_processor.final_json_folder}")
    print(f"📁 처리되지 않은 PDF 파일 위치: {post_processor.not_processed_folder}")
    
except Exception as e:
    print(f"❌ 후처리 중 오류 발생: {str(e)}")
    print("💡 오류 해결 방법:")
    print("  - CSV 파일이 올바른 위치에 있는지 확인")
    print("  - 필수 컬럼 (final_fake_code, fake_grade, maker pn)이 있는지 확인")
    print("  - JSON 파일이 올바르게 생성되었는지 확인")

In [ ]:
#@title 🔍 7단계: 최종 벡터 스토어 생성 { display-mode: "form" }
#@markdown 후처리가 완료된 JSON 파일들로부터 벡터 스토어를 생성합니다.

from src.vectorstore_manager import VectorstoreManager
import json
from pathlib import Path

# param.json 상태 확인 (디버깅)
import json
from pathlib import Path

param_path = Path('misc/param.json')
if param_path.exists():
    with open(param_path, 'r') as f:
        current_params = json.load(f)
    print(f"🔍 param.json의 vectorstore 경로: {current_params.get('vectorstore', {}).get('folders', {}).get('vectorstore_folder', 'NOT_FOUND')}")
else:
    print("❌ param.json 파일이 없습니다!")

# Vectorstore Manager 초기화 (param.json에서 경로 자동 인식)
print("🔄 VectorstoreManager 초기화 중...")

# 어떤 모듈이 import되는지 확인
from src.vectorstore_manager import VectorstoreManager
import inspect
print(f"🔍 VectorstoreManager 위치: {inspect.getfile(VectorstoreManager)}")

vectorstore_manager = VectorstoreManager()

# 초기화 후 실제 사용되는 경로 확인
print(f"🔍 VectorstoreManager가 사용하는 경로: {vectorstore_manager.params['folders']['vectorstore_folder']}")
print(f"🔍 VectorstoreManager model_name: {getattr(vectorstore_manager, 'model_name', 'NOT_FOUND')}")

# param.json에서 경로 읽어오기
param_path = Path('misc/param.json')
with open(param_path, 'r') as f:
    params = json.load(f)

result_json_folder = Path(params['pdf_processing']['folders']['result_json_folder'])

# 최종 처리된 JSON 파일 로드
final_json_folder = result_json_folder / 'final_json'
json_files = list(final_json_folder.glob("*.json"))

if not json_files:
    print("❌ 최종 처리된 JSON 파일이 없습니다.")
    print("💡 6단계 후처리를 먼저 실행해주세요.")
    
    # 대안으로 원본 JSON 파일 확인
    original_json_folder = result_json_folder
    original_files = list(original_json_folder.glob("*_combined.json"))
    
    if original_files:
        print(f"\n🔍 원본 JSON 파일 발견: {len(original_files)}개")
        print("원본 파일로 벡터 스토어를 생성하시겠습니까?")
        json_files = original_files
        final_json_folder = original_json_folder
    else:
        print("❌ 처리할 JSON 파일이 없습니다. 5단계와 6단계를 먼저 실행해주세요.")

if json_files:
    print(f"📄 발견된 JSON 파일: {len(json_files)}개")
    
    # JSON 데이터 로드
    json_data = []
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                json_data.append(data)
            print(f"✅ {json_file.name} 로드 완료")
        except Exception as e:
            print(f"❌ {json_file.name} 로드 실패: {str(e)}")
    
    if json_data:
        print(f"\n🔄 벡터 스토어 생성 중... (총 {len(json_data)}개 파일)")
        try:
            # 벡터 스토어 생성
            vectorstore = vectorstore_manager.create_vectorstore(json_data)
            
            # 벡터 스토어 저장
            vectorstore_name = "datasheet_vectors_final"
            vectorstore_manager.save_vectorstore(vectorstore, vectorstore_name)
            
            print("✅ 벡터 스토어 생성 및 저장 완료!")
            print(f"💾 저장 위치: {vectorstore_folder}/{vectorstore_name}")
            print(f"📊 처리된 문서 수: {len(json_data)}")
            
            # 벡터 스토어 정보 출력
            print("\n📋 벡터 스토어 정보:")
            try:
                model_name = getattr(vectorstore_manager, 'model_name', 'Unknown')
                print(f"• 사용된 임베딩 모델: {model_name}")
            except Exception as e:
                print(f"• 사용된 임베딩 모델: 확인 실패 ({str(e)})")
            print(f"• 저장 형식: FAISS")
            print(f"• 디바이스: CPU 전용")
            
        except Exception as e:
            print(f"❌ 벡터 스토어 생성 실패: {str(e)}")
            print("💡 오류 해결 방법:")
            print("  - HuggingFace 토큰이 올바르게 설정되었는지 확인")
            print("  - GPU 메모리가 충분한지 확인")
            print("  - JSON 파일 형식이 올바른지 확인")
    else:
        print("❌ 로드된 JSON 데이터가 없습니다.")


In [ ]:
#@title 📋 8단계: 처리 완료 점검 및 요약 { display-mode: "form" }
#@markdown 전체 처리 과정을 실제로 점검하고 요약 정보를 제공합니다.

import os
import json
from pathlib import Path

print("🔍 PDF 데이터시트 전처리 결과 점검 중...")
print("=" * 60)

# 점검 결과를 저장할 리스트
check_results = []
total_score = 0
max_score = 0

def add_check(name, passed, details="", critical=False):
    global total_score, max_score
    score = 2 if critical else 1
    max_score += score
    if passed:
        total_score += score
        status = "✅"
    else:
        status = "❌" if critical else "⚠️"
    
    check_results.append({
        'name': name,
        'status': status,
        'passed': passed,
        'details': details,
        'critical': critical
    })

# 1. 원본 PDF 파일 점검
print("\n1️⃣ 원본 PDF 파일 점검...")
if pdf_folder.exists():
    pdf_files = list(pdf_folder.glob("*.pdf"))
    add_check("원본 PDF 파일", len(pdf_files) > 0, f"{len(pdf_files)}개 파일 발견", critical=True)
else:
    add_check("원본 PDF 파일", False, "PDF 폴더가 존재하지 않음", critical=True)

# 2. 처리된 JSON 파일 점검
print("2️⃣ 처리된 JSON 파일 점검...")
prep_json_folder = result_json_folder
if prep_json_folder.exists():
    combined_json_files = list(prep_json_folder.glob("*_combined.json"))
    add_check("처리된 JSON 파일", len(combined_json_files) > 0, f"{len(combined_json_files)}개 파일 생성", critical=True)
    
    # JSON 파일 내용 샘플 점검
    if combined_json_files:
        try:
            with open(combined_json_files[0], 'r', encoding='utf-8') as f:
                sample_data = json.load(f)
            
            has_required_fields = all(key in sample_data for key in ['filename', 'category_chunks', 'page_summaries'])
            add_check("JSON 구조 유효성", has_required_fields, 
                     "필수 필드 (filename, category_chunks, page_summaries) 확인")
        except Exception as e:
            add_check("JSON 구조 유효성", False, f"JSON 파싱 오류: {str(e)}")
else:
    add_check("처리된 JSON 파일", False, "prep_json 폴더가 존재하지 않음", critical=True)

# 3. 후처리된 최종 JSON 파일 점검
print("3️⃣ 후처리된 최종 JSON 파일 점검...")
final_json_folder = result_json_folder / 'final_json'
if final_json_folder.exists():
    final_json_files = list(final_json_folder.glob("*.json"))
    add_check("최종 JSON 파일", len(final_json_files) > 0, f"{len(final_json_files)}개 파일 생성")
    
    # 정규화된 필드 점검
    if final_json_files:
        try:
            with open(final_json_files[0], 'r', encoding='utf-8') as f:
                sample_final = json.load(f)
            
            has_normalized_fields = 'part number' in sample_final or 'grade' in sample_final.get('metadata', {})
            add_check("필드명 정규화", has_normalized_fields, "part number, grade 필드 정규화 확인")
        except Exception as e:
            add_check("필드명 정규화", False, f"최종 JSON 점검 오류: {str(e)}")
else:
    add_check("최종 JSON 파일", False, "final_json 폴더가 존재하지 않음")

# 4. 벡터 스토어 점검
print("4️⃣ 벡터 스토어 점검...")
vectorstore_path = vectorstore_folder / 'datasheet_vectors_final'
if vectorstore_path.exists():
    # FAISS 인덱스 파일들 확인
    index_file = vectorstore_path / 'index.faiss'
    pkl_file = vectorstore_path / 'index.pkl'
    
    faiss_files_exist = index_file.exists() and pkl_file.exists()
    add_check("벡터 스토어 파일", faiss_files_exist, 
             f"FAISS 인덱스 파일 {'존재' if faiss_files_exist else '누락'}", critical=True)
    
    # 벡터 스토어 로드 테스트
    if faiss_files_exist:
        try:
            from src.vectorstore_manager import VectorstoreManager
            test_vm = VectorstoreManager()
            test_vectorstore = test_vm.load_vectorstore('datasheet_vectors_final')
            
            # 간단한 검색 테스트
            test_results = test_vectorstore.similarity_search("test", k=1)
            add_check("벡터 스토어 로드", True, f"로드 성공, {len(test_results)}개 검색 결과")
            
            # 벡터 스토어 정보 확인
            if hasattr(test_vectorstore, 'index') and hasattr(test_vectorstore.index, 'ntotal'):
                doc_count = test_vectorstore.index.ntotal
                add_check("벡터 스토어 내용", doc_count > 0, f"총 {doc_count}개 문서 벡터화됨")
            else:
                add_check("벡터 스토어 내용", False, "문서 개수 확인 불가")
                
        except Exception as e:
            add_check("벡터 스토어 로드", False, f"로드 실패: {str(e)}")
else:
    add_check("벡터 스토어 파일", False, "datasheet_vectors_final 폴더가 존재하지 않음", critical=True)

# param.json에서 경로 읽어오기
param_path = Path('misc/param.json')
with open(param_path, 'r') as f:
    params = json.load(f)

pdf_folder = Path(params['pdf_processing']['folders']['pdf_folder'])
result_json_folder = Path(params['pdf_processing']['folders']['result_json_folder'])

# 5. 처리되지 않은 PDF 점검
print("5️⃣ 처리되지 않은 PDF 점검...")
not_processed_folder = pdf_folder / 'notprocessed'
if not_processed_folder.exists():
    not_processed_files = list(not_processed_folder.glob("*.pdf"))
    add_check("처리되지 않은 PDF", True, f"{len(not_processed_files)}개 파일 (적을수록 좋음)")
else:
    add_check("처리되지 않은 PDF", True, "notprocessed 폴더 없음 (모든 파일 처리됨)")

# 결과 출력
print("\n" + "=" * 60)
print("📊 점검 결과 요약:")
print("=" * 60)

for result in check_results:
    details_str = f" - {result['details']}" if result['details'] else ""
    print(f"{result['status']} {result['name']}{details_str}")

# 점수 계산
score_percentage = (total_score / max_score * 100) if max_score > 0 else 0
print(f"\n🎯 전체 점수: {total_score}/{max_score} ({score_percentage:.1f}%)")

# 상태 판정
critical_failures = [r for r in check_results if not r['passed'] and r['critical']]
if critical_failures:
    print("❌ 치명적 오류가 발견되었습니다:")
    for failure in critical_failures:
        print(f"   • {failure['name']}: {failure['details']}")
    print("\n💡 위 문제들을 해결한 후 다시 실행해주세요.")
elif score_percentage >= 80:
    print("✅ 전처리 파이프라인이 성공적으로 완료되었습니다!")
    print("\n📁 주요 파일 위치:")
    print(f"• 원본 PDF: {pdf_folder}")
    print(f"• 최종 JSON: {final_json_folder}")
    print(f"• 벡터 스토어: {vectorstore_path}")
    
    print("\n🚀 이제 챗봇 시스템에서 벡터 스토어를 사용할 수 있습니다!")
else:
    print("⚠️ 일부 단계에서 문제가 발견되었습니다.")
    print("💡 경고사항들을 검토하고 필요시 해당 단계를 다시 실행해주세요.")

print("\n" + "=" * 60)
